In [1]:
import os
from dotenv import load_dotenv
import requests
import json
import time
from functools import reduce
import operator
import pandas as pd

The list of cities I got from the database filtering is the following:

In [2]:
cities = ['San Francisco, CA, USA', 'Ellensburg, WA, USA', 'Brooklyn, NY, USA']

In [3]:
cities_short = [i.split(",")[0] for i in cities]

In [4]:
if not os.path.exists(f"../newdata"):
    os.mkdir(f"../newdata")

In [5]:
load_dotenv()

True

In [6]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [7]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    time.sleep(3)
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [8]:
sanfrancisco = {'type': 'Point', 'coordinates':[37.778480215129015, -122.40559118863916]}
ellensburg =  {'type': 'Point', 'coordinates':[46.996517823587766, -120.54550769638885]}
brooklyn =  {'type': 'Point', 'coordinates':[40.692147496548735, -73.98495174207916]}

In [9]:
queries = ["vegan", "starbucks", "school","train station subway", "basketball court" ]

In [10]:
def getFromDict(diccionario,mapa):
    return reduce (operator.getitem,mapa,diccionario)

In [11]:
def get_data (latitude, longitude, query):
    d = {}
    time.sleep(2)
    parametres = {"client_id" : client_id,
              "client_secret" : client_secret,
              "v": "20180323",
              "ll": f"{latitude},{longitude}", 
              "query":query,
              "limit": 100}
    time.sleep(1)

    resp = requests.get(url= 'https://api.foursquare.com/v2/venues/search', params=parametres).json()

    return resp

In [12]:
def respven(dic):
    return dic['response']['venues']

In [13]:
def extraetodo(json):
    todo = {"name": ["name"], "lat": ["location", "lat"], "lon": ["location", "lng"], "distance":['location', 'distance']} 
    total = []
    for elemento in json:
        dic = {key: getFromDict(elemento, value) for key,value in todo.items()}
        dic["location"] = {'type': 'Point', 'coordinates': [dic["lat"], dic["lon"]]}
        total.append(dic)
    return total

In [14]:
def fin(city, n):
    return extraetodo(respven(get_data(city['coordinates'][0], city['coordinates'][1], queries[n])))

In [15]:
def todf(fincity):
    fordf = []
    for dic in fincity:
            for i in dic:
                todf = {}
                todf["name"] = getFromDict(i,["name"])
                todf["lat"] = getFromDict(i,["lat"])
                todf["lon"] = getFromDict(i,["lon"])
                todf["location"] = getFromDict(i,["location"])
                todf["category"] = getFromDict(i,["category"])
                todf["distance"] = getFromDict (i,["distance"])
                fordf.append(todf)
    df = pd.DataFrame(fordf)
    return df

In [16]:
def dropbydist(df, dis, cat):
    return df.drop(df[(df.distance > dis) & (df.category == cat)].index, inplace=True)

In [17]:
category = ["vegan_venues", "starbucks", "schools","train_stations", "basketball_courts" ]

# San Fracisco

In [18]:
vegsanfrancisco = fin(sanfrancisco, 0)
for  i in  vegsanfrancisco:
    i['category']= 'vegan_venues'
    
stasanfrancisco = fin(sanfrancisco, 1)
for  i in  stasanfrancisco:
    i['category']= 'starbucks'
    
schsanfrancisco = fin(sanfrancisco, 2)
for  i in  schsanfrancisco:
    i['category']= 'schools'

trasanfrancisco = fin(sanfrancisco, 3)
for  i in  trasanfrancisco:
    i['category']= 'train_stations'

bassanfrancisco = fin(sanfrancisco, 4)
for  i in  bassanfrancisco:
    i['category']= 'basketball_courts'

In [19]:
finsanfrancisco = [vegsanfrancisco,stasanfrancisco, schsanfrancisco, trasanfrancisco, bassanfrancisco]

In [20]:
dfsanfrancisco = todf(finsanfrancisco)

In [21]:
dropbydist(dfsanfrancisco, 1500, category[0])
dropbydist(dfsanfrancisco, 1300, category[1])
dropbydist(dfsanfrancisco, 3000, category[2])
dropbydist(dfsanfrancisco, 2500, category[3])
dropbydist(dfsanfrancisco, 2000, category[4])

In [36]:
dfsanfrancisco.to_csv(f"../newdata/dfsanfran.csv", index = False)

# Ellensburg

In [24]:
vegell = fin(ellensburg, 0)
for  i in  vegell:
    i['category'] = 'vegan_venues'
    
staell = fin(ellensburg, 1)
for  i in  staell:
    i['category'] = 'starbucks'

schell = fin(ellensburg, 2)
for  i in  schell:
    i['category'] = 'schools'

traell = fin(ellensburg, 3)
for  i in  traell:
    i['category'] = 'train_stations'

basell = fin(ellensburg, 4)
for  i in  basell:
    i['category'] = 'basketball_courts'

In [25]:
finellensburg = [vegell, staell, schell, traell, basell]

In [26]:
dfellensburg = todf(finellensburg)

In [27]:
dropbydist(dfellensburg, 1500, category[0])
dropbydist(dfellensburg, 1300, category[1])
dropbydist(dfellensburg, 3000, category[2])
dropbydist(dfellensburg, 2500, category[3])
dropbydist(dfellensburg, 2000, category[4])

In [28]:
dfellensburg

,name,lat,lon,location,category,distance
0,Starbucks,46.996148,-120.543582,"{'type': 'Point', 'coordinates': [46.996148021...",starbucks,151
1,Starbucks,46.991140,-120.551027,"{'type': 'Point', 'coordinates': [46.99114, -1...",starbucks,730
2,Starbucks,47.001745,-120.534619,"{'type': 'Point', 'coordinates': [47.001745253...",starbucks,1010
10,Ellensburg High School,46.991577,-120.545212,"{'type': 'Point', 'coordinates': [46.991577148...",schools,550
11,Ellensburg washington school of karate,46.995197,-120.545502,"{'type': 'Point', 'coordinates': [46.995197296...",schools,147
12,Morgan Middle School,46.992395,-120.543012,"{'type': 'Point', 'coordinates': [46.992395107...",schools,496
13,Ellensburg Montessori School,46.997812,-120.536559,"{'type': 'Point', 'coordinates': [46.997812, -...",schools,694
14,Lincoln Elementary School,46.990882,-120.539754,"{'type': 'Point', 'coordinates': [46.990882, -...",schools,764
15,Ellensburg Christian School,46.988830,-120.541613,"{'type': 'Point', 'coordinates': [46.988829824...",schools,905
44,Northern Pacific Train Station,46.993900,-120.554924,"{'type': 'Point', 'coordinates': [46.993900299...",train_stations,772


In [29]:
dfellensburg.to_csv(f"../newdata/dfellensburg.csv", index = False)

# Brooklyn

In [30]:
vegbro = fin(brooklyn, 0)
for  i in  vegbro:
    i['category'] = 'vegan_venues'

stabro = fin(brooklyn, 1)
for  i in  stabro:
    i['category'] = 'starbucks'
    
schbro = fin(brooklyn, 2)
for  i in  schbro:
    i['category'] = 'schools'
    
trabro = fin(brooklyn, 3)
for  i in  trabro:
    i['category'] = 'train_stations'

basbro = fin(brooklyn, 4)
for  i in  basbro:
    i['category'] = 'basketball_courts'

In [31]:
finbrooklyn = [vegbro, stabro, schbro, trabro, basbro]

In [32]:
dfbrooklyn = todf(finbrooklyn)

In [33]:
dropbydist(dfbrooklyn, 1500, category[0])
dropbydist(dfbrooklyn, 1300, category[1])
dropbydist(dfbrooklyn, 3000, category[2])
dropbydist(dfbrooklyn, 2500, category[3])
dropbydist(dfbrooklyn, 2000, category[4])

In [34]:
dfbrooklyn.to_csv(f"../newdata/dfbrooklyn.csv", index = False)

In [35]:
dfbrooklyn

,name,lat,lon,location,category,distance
0,Try Vegan,40.697583,-73.981548,"{'type': 'Point', 'coordinates': [40.697582909...",vegan_venues,669
12,Betty Bakery,40.686385,-73.983350,"{'type': 'Point', 'coordinates': [40.686384555...",vegan_venues,655
36,LuAnne's Wild Ginger,40.689644,-73.969824,"{'type': 'Point', 'coordinates': [40.689644, -...",vegan_venues,1306
39,Wild Ginger,40.687898,-73.989730,"{'type': 'Point', 'coordinates': [40.687897607...",vegan_venues,621
50,Starbucks,40.694019,-73.986800,"{'type': 'Point', 'coordinates': [40.694019, -...",starbucks,260
...,...,...,...,...,...,...
188,MTA Subway - East Broadway (F),40.713973,-73.990125,"{'type': 'Point', 'coordinates': [40.713973175...",train_stations,2468
190,MTA Subway - Smith/9th St (F/G),40.674128,-73.996774,"{'type': 'Point', 'coordinates': [40.674127996...",train_stations,2240
193,MTA Subway - Lafayette Ave (C),40.686082,-73.973891,"{'type': 'Point', 'coordinates': [40.686081961...",train_stations,1152
200,State Street Basketball Court,40.691670,-73.998587,"{'type': 'Point', 'coordinates': [40.69167, -7...",basketball_courts,1152
